In [1]:
import pandas as pd
# 여러 변수 출력 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
df_bus = pd.read_csv('bus_station_coordinate.csv', encoding='cp949')

In [3]:
df_sub = pd.read_csv('subway_station_coordinate.csv')

In [18]:
df_bus.head()
len(df_bus)
df_sub.head()

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,버스도착정보안내기_설치_여부
0,717104358,계수사거리(경유),가상정류장,0.0,126.817473,37.452473,미설치
1,500000288,행사장입구,마을버스,25965.0,126.691556,37.321742,미설치
2,500000287,행사장입구,마을버스,25095.0,126.691728,37.321942,미설치
3,500000286,아남중전기,일반차로,25963.0,126.710485,37.319934,미설치
4,500000285,3Z,일반차로,25964.0,126.707130,37.320566,미설치


69942

,line,name,code,lat,lng,distance
0,01호선,녹양,1908.0,37.759380,127.042292,222.0
1,01호선,남영,1002.0,37.541021,126.971300,215.0
2,01호선,용산,1003.0,37.529849,126.964561,215.0
3,01호선,노량진,1004.0,37.514219,126.942454,216.0
4,01호선,대방,1005.0,37.513342,126.926382,217.0


In [5]:
!pip install haversine

In [6]:
from haversine import haversine

In [30]:
df_bus_one = df_bus.loc[60000]
bus_spot = (df_bus_one['경도'], df_bus_one['위도'])
bus_spot

df_sub['distance'] = 0
for i in range(len(df_sub)):
    df_sub['distance'][i] = haversine(bus_spot, (df_sub['lat'][i], df_sub['lng'][i]))



(37.486881, 126.947837)

<ipython-input-30-5d10a807e06f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['distance'][i] = haversine(bus_spot, (df_sub['lat'][i], df_sub['lng'][i]))


line              02호선
name             서울대입구
code             228.0
lat          37.481247
lng         126.952739
distance           0.0
Name: 111, dtype: object

In [31]:
near_sub = df_sub.loc[df_sub['distance'].idxmin()]
near = pd.concat([df_bus_one, near_sub])
near_sub
near

line              02호선
name             서울대입구
code             228.0
lat          37.481247
lng         126.952739
distance           0.0
Name: 111, dtype: object

정류장_ID              120000149
정류장_명칭                   현대시장
정류장_유형                   일반차로
정류장_번호                21251.0
위도                 126.947837
경도                  37.486881
버스도착정보안내기_설치_여부         설치   
line                     02호선
name                    서울대입구
code                    228.0
lat                 37.481247
lng                126.952739
distance                  0.0
dtype: object

In [ ]:
new_df = pd.DataFrame(columns=near.keys())
new_df

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,버스도착정보안내기_설치_여부,line,name,code,lat,lng,distance


In [46]:
!pip install tqdm

     |████████████████████████████████| 76 kB 8.1 MB/s             


In [47]:
import time
from tqdm import tqdm

In [48]:

for j in tqdm(range(len(df_bus))):

    df_bus_one = df_bus.loc[j]
    bus_spot = (df_bus_one['경도'], df_bus_one['위도']) #버스 지점 하나

    #거리계산 시작
    df_sub['distance'] = 0 #거리 열 추가
    for i in range(len(df_sub)):
        df_sub['distance'][i] = haversine(bus_spot, (df_sub['lat'][i], df_sub['lng'][i])) #모든 구간 거리계산 완료

    try:
        #그 중에 가장 가까운 행(내림차순으로 가장 첫번째 행)
        near_sub = df_sub.loc[df_sub['distance'].idxmin()]
        
    except:
        near_sub = pd.Series({"line":'','name':'','code':'','lat':'','lng':'','distance':''})

    near = pd.concat([df_bus_one, near_sub]) #버스 정보 + 가장 가까운 지하철역 정보

    #비어있던 new_df에 한줄씩 추가
    new_df.loc[j] = near

  0%|          | 0/69942 [00:00<?, ?it/s]<ipython-input-48-269a7340a9b6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['distance'][i] = haversine(bus_spot, (df_sub['lat'][i], df_sub['lng'][i])) #모든 구간 거리계산 완료
/opt/homebrew/Caskroom/miniforge/base/envs/mulcam_ds/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 55%|█████▌    | 38578/69942 [17:50<15:00, 34.81it/s]

In [43]:
# line              01호선
# name                역곡
# code            1803.0
# lat          37.485178
# lng         126.811502
# distance           3.0
# Name: 49, dtype: object

near_sub = pd.Series({"line":'','name':'','code':'','lat':'','lng':'','distance':''})
near_sub

line        
name        
code        
lat         
lng         
distance    
dtype: object

In [33]:
new_df

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,버스도착정보안내기_설치_여부,line,name,code,lat,lng,distance
0,717104358,계수사거리(경유),가상정류장,0.0,126.817473,37.452473,미설치,01호선,녹양,1908.0,37.759380,127.042292,39.0
1,500000288,행사장입구,마을버스,25965.0,126.691556,37.321742,미설치,01호선,녹양,1908.0,37.759380,127.042292,57.0
2,500000287,행사장입구,마을버스,25095.0,126.691728,37.321942,미설치,01호선,녹양,1908.0,37.759380,127.042292,57.0
3,500000286,아남중전기,일반차로,25963.0,126.710485,37.319934,미설치,01호선,녹양,1908.0,37.759380,127.042292,56.0
4,500000285,3Z,일반차로,25964.0,126.707130,37.320566,미설치,01호선,녹양,1908.0,37.759380,127.042292,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63295,114000112,신곡시장.금빛새마을금고,일반차로,15215.0,126.843743,37.519643,설치,02호선,신정네거리,249.0,37.520074,126.852912,0.0
63296,114000111,동방아파트.신월종합사회복지관,일반차로,15214.0,126.846290,37.521426,미설치,02호선,신정네거리,249.0,37.520074,126.852912,0.0
63297,114000110,동방아파트,일반차로,15213.0,126.846325,37.521882,미설치,02호선,신정네거리,249.0,37.520074,126.852912,0.0
63298,114000109,신정네거리.신정119안전센터,일반차로,15212.0,126.850730,37.522023,설치,02호선,신정네거리,249.0,37.520074,126.852912,0.0


In [ ]:
near = pd.Series({""})

In [ ]:
new_df.to_csv('거리계산완료.csv')